In [2]:
using Catlab
using Catlab.CategoricalAlgebra
using Catlab.Graphics
using Catlab.Programs
using CombinatorialSpaces
using CombinatorialSpaces.ExteriorCalculus
using Decapodes
using MultiScaleArrays
using OrdinaryDiffEq
using MLStyle
using Distributions
using LinearAlgebra
# using GLMakie
using Logging
using CairoMakie 

In [3]:
using GeometryBasics: Point3
Point3D = Point3{Float64}


Point3{Float64} (alias for Point{3, Float64})

In [4]:
# Begin Navier Stokes

# Navier Stokes example
DiffusionExprBody = quote
  (T, Ṫ)::Form0{X}
  ϕ::DualForm1{X}
  k::Constant{X}
  # Fick's first law
  ϕ ==  ⋆₁(k*d₀(T))
  # Diffusion equation
  Ṫ == ⋆₀⁻¹(dual_d₁(ϕ))
end

quote
    #= In[4]:5 =#
    (T, Ṫ)::Form0{X}
    #= In[4]:6 =#
    ϕ::DualForm1{X}
    #= In[4]:7 =#
    k::Constant{X}
    #= In[4]:9 =#
    ϕ == (⋆₁)(k * d₀(T))
    #= In[4]:11 =#
    Ṫ == (⋆₀⁻¹)(dual_d₁(ϕ))
end

In [5]:
Diffusion = SummationDecapode(parse_decapode(DiffusionExprBody))
to_graphviz(Diffusion)
to_graphviz(Diffusion, graph_attrs=Dict(:rankdir => "LR"))

AdvectionExprBody = quote
  (M,V)::Form1{X}  #  M = ρV
  (ρ, p, T, Ṫ)::Form0{X}
  V == M/avg₀₁(ρ)
  ρ == p / R₀(T)
  Ṫ == neg₀(⋆₀⁻¹(L₀(V, ⋆₀(T))))
end

quote
    #= In[5]:6 =#
    (M, V)::Form1{X}
    #= In[5]:7 =#
    (ρ, p, T, Ṫ)::Form0{X}
    #= In[5]:8 =#
    V == M / avg₀₁(ρ)
    #= In[5]:9 =#
    ρ == p / R₀(T)
    #= In[5]:10 =#
    Ṫ == neg₀((⋆₀⁻¹)(L₀(V, (⋆₀)(T))))
end

In [6]:
AdvectionExprBody = quote
  (M,V)::Form1{X}  #  M = ρV
  (ρ, p, T, Ṫ)::Form0{X}
  V == M/avg₀₁(ρ)
  ρ == p / R₀(T)
  Ṫ == neg₀(⋆₀⁻¹(L₀(V, ⋆₀(T))))
end

Advection = SummationDecapode(parse_decapode(AdvectionExprBody))
to_graphviz(Advection)
to_graphviz(Advection, graph_attrs=Dict(:rankdir => "LR"))

SuperpositionExprBody = quote
  (T, Ṫ, Ṫ₁, Ṫₐ)::Form0{X}
  Ṫ == Ṫ₁ + Ṫₐ
  ∂ₜ(T) == Ṫ 
end
Superposition = SummationDecapode(parse_decapode(SuperpositionExprBody))
to_graphviz(Superposition)
to_graphviz(Superposition, graph_attrs=Dict(:rankdir => "LR"))

compose_continuity = @relation () begin
  diffusion(T, Ṫ₁)
  advection(M, ρ, P, T, Ṫₐ)
  superposition(T, Ṫ, Ṫ₁, Ṫₐ)
end
#to_graphviz(compose_continuity, junction_labels=:variable, box_labels=:name, prog="neato")
to_graphviz(compose_continuity, junction_labels=:variable, box_labels=:name, prog="circo")

continuity_cospan = oapply(compose_continuity,
                [Open(Diffusion, [:T, :Ṫ]),
                 Open(Advection, [:M, :ρ, :p, :T, :Ṫ]),
                 Open(Superposition, [:T, :Ṫ, :Ṫ₁, :Ṫₐ])])

continuity = apex(continuity_cospan)

Var,type,name
1,Form0,T
2,Form0,Ṫ₁
3,DualForm1,diffusion_ϕ
4,Constant,diffusion_k
5,infer,diffusion_•1
6,infer,diffusion_•2
7,infer,diffusion_•3
8,Form1,M
9,Form1,advection_V
10,Form0,ρ


In [10]:
using Catlab
using CombinatorialSpaces
using CombinatorialSpaces.ExteriorCalculus
using Decapodes
using MultiScaleArrays
using OrdinaryDiffEq
using MLStyle
using Distributions
using LinearAlgebra
# using GLMakie
using Logging
using CairoMakie

using GeometryBasics: Point3
Point3D = Point3{Float64}


PressureFlow = quote
  # state variables
  V::Form1{X}
  P::Form0{X}

  # derived quantities
  ΔV::Form1{X}
  ∇P::Form0{X}
  ΔP::Form0{X}
  ϕₚ::Form1{X}

  # tanvars
  V̇::Form1{X}
  Ṗ::Form0{X}
  ∂ₜ(V) == V̇
  ∂ₜ(P) == Ṗ
  
  ∇P == d₀(P)
  ΔV == Δ₁(V)
  ΔP == Δ₀(P)

  V̇  == α(∇P) + μ(ΔV)
  ϕₚ == γ(-(L₀(V, P))) 
  Ṗ == β(Δ₀(P)) + ∘(dual_d₁,⋆₀⁻¹)(ϕₚ)
  # Ṗ  == ϕₚ
end

@time begin
pf = SummationDecapode(parse_decapode(PressureFlow))

flatten_form(vfield::Function, mesh) =  ♭(mesh, DualVectorField(vfield.(mesh[triangle_center(mesh),:dual_point])))

function generate(sd, my_symbol; hodge=GeometricHodge())
  i0 = (v,x) -> ⋆(1, sd, hodge=hodge)*wedge_product(Tuple{0,1}, sd, v, inv_hodge_star(0,sd, hodge=DiagonalHodge())*x)
  op = @match my_symbol begin
    :k => x->2000x
    :μ => x->-0.0001x
    # :μ => x->-2000x
    :α => x->0*x
    :β => x->2000*x
    :γ => x->1*x
    :⋆₀ => x->⋆(0,sd,hodge=hodge)*x
    :⋆₁ => x->⋆(1, sd, hodge=hodge)*x
    :⋆₀⁻¹ => x->inv_hodge_star(0,sd, x; hodge=hodge)
    :⋆₁⁻¹ => x->inv_hodge_star(1,sd,hodge=hodge)*x
    :d₀ => x->d(0,sd)*x
    :d₁ => x->d(1,sd)*x
    :dual_d₀ => x->dual_derivative(0,sd)*x
    :dual_d₁ => x->dual_derivative(1,sd)*x
    :∧₀₁ => (x,y)-> wedge_product(Tuple{0,1}, sd, x, y)
    :plus => (+)
    :(-) => x-> -x
    # :L₀ => (v,x)->dual_derivative(1, sd)*(i0(v, x))
    :L₀ => (v,x)->begin
      # dual_derivative(1, sd)*⋆(1, sd)*wedge_product(Tuple{1,0}, sd, v, x)
      ⋆(1, sd; hodge=hodge)*wedge_product(Tuple{1,0}, sd, v, x)
    end
    :i₀ => i0 
    :Δ₀ => x -> begin # dδ
      δ(1, sd, d(0, sd)*x, hodge=hodge) end
    # :Δ₀ => x -> begin # d ⋆ d̃ ⋆⁻¹
    #   y = dual_derivative(1,sd)*⋆(1, sd, hodge=hodge)*d(0,sd)*x
    #   inv_hodge_star(0,sd, y; hodge=hodge)
    # end
    :Δ₁ => x -> begin # dδ + δd
      δ(2, sd, d(1, sd)*x, hodge=hodge) + d(0, sd)*δ(1, sd, x, hodge=hodge)
    end

    # :Δ₁ => x -> begin # d ⋆ d̃ ⋆⁻¹ + ⋆ d̃ ⋆ d
    #   y = dual_derivative(0,sd)*⋆(2, sd, hodge=hodge)*d(1,sd)*x
    #   inv_hodge_star(2,sd, y; hodge=hodge) 
    #   z = d(0, sd) * inverse_hode_star(2, sd, dual_derivative(1, sd)*⋆(1,sd, hodge=hodge)*x; hodge=hodge)
    #   return y + z
    # end
    :debug => (args...)->begin println(args[1], length.(args[2:end])) end
    x=> error("Unmatched operator $my_symbol")
  end
  # return (args...) -> begin println("applying $my_symbol"); println("arg length $(length.(args))"); op(args...);end
  return (args...) ->  op(args...)
end

end

  0.007368 seconds (12.24 k allocations: 832.607 KiB)


generate (generic function with 1 method)

In [12]:
#include("coordinates.jl")
#include("spherical_meshes.jl")
@time begin
radius = 6371+90

#primal_earth = loadmesh(ThermoIcosphere())
primal_earth = loadmesh(Icosphere(3, radius))
nploc = argmax(x -> x[3], primal_earth[:point])

orient!(primal_earth)
earth = EmbeddedDeltaDualComplex2D{Bool,Float64,Point3D}(primal_earth)
subdivide_duals!(earth, Circumcenter())

physics = SummationDecapode(parse_decapode(PressureFlow))
gensim(expand_operators(physics), [:P, :V])
sim = eval(gensim(expand_operators(physics), [:P, :V]))

fₘ = sim(earth, generate)

begin
  vmag = 500
  # velocity(p) = vmag*ϕhat(p)
  velocity(p) = TangentBasis(CartesianPoint(p))((vmag/4, vmag/4, 0))
  # velocity(p) = TangentBasis(CartesianPoint(p))((vmag/4, -vmag/4, 0))

# visualize the vector field
  ps = earth[:point]
  ns = ((x->x) ∘ (x->Vec3f(x...))∘velocity).(ps)
#   arrows(
#       ps, ns, fxaa=true, # turn on anti-aliasing
#       linecolor = :gray, arrowcolor = :gray,
#       linewidth = 20.1, arrowsize = 20*Vec3f(3, 3, 4),
#       align = :center, axis=(type=Axis3,)
#   )
end

begin
v = flatten_form(velocity, earth)
c_dist = MvNormal([radius/√(2), radius/√(2)], 20*[1, 1])
c = 100*[pdf(c_dist, [p[1], p[2]]) for p in earth[:point]]

theta_start = 45*pi/180
phi_start = 0*pi/180
x = radius*cos(phi_start)*sin(theta_start)
y = radius*sin(phi_start)*sin(theta_start)
z = radius*cos(theta_start)
c_dist₁ = MvNormal([x, y, z], 20*[1, 1, 1])
c_dist₂ = MvNormal([x, y, -z], 20*[1, 1, 1])

c_dist = MixtureModel([c_dist₁, c_dist₂], [0.6,0.4])

c = 100*[pdf(c_dist, [p[1], p[2], p[3]]) for p in earth[:point]]


u₀ = construct(PhysicsState, [VectorForm(c), VectorForm(collect(v))],Float64[], [:P, :V])
mesh(primal_earth, color=findnode(u₀, :P), colormap=:plasma)
tₑ = 30.0

end
    
end

 37.012097 seconds (61.29 M allocations: 3.228 GiB, 1.86% gc time, 98.28% compilation time: 13% of which was recompilation)


30.0

In [ ]:
    
@time begin
        
@info("Precompiling Solver")
prob = ODEProblem(fₘ,u₀,(0,1e-4))
soln = solve(prob, Tsit5())
soln.retcode != :Unstable || error("Solver was not stable")
@info("Solving")
prob = ODEProblem(fₘ,u₀,(0,tₑ))
soln = solve(prob, Tsit5())
@info("Done")
end
    
end

┌ Info: Precompiling Solver
└ @ Main In[13]:4
┌ Info: Solving
└ @ Main In[13]:8


In [1]:
begin
mass(soln, t, mesh, concentration=:P) = sum(⋆(0, mesh)*findnode(soln(t), concentration))

@show extrema(mass(soln, t, earth, :P) for t in 0:tₑ/150:tₑ)
end
mesh(primal_earth, color=findnode(soln(0), :P), colormap=:jet)
mesh(primal_earth, color=findnode(soln(0) - soln(tₑ), :P), colormap=:jet)
begin
# Plot the result
times = range(0.0, tₑ, length=150)
colors = [findnode(soln(t), :P) for t in times]

# Initial frame
# fig, ax, ob = mesh(primal_earth, color=colors[1], colorrange = extrema(vcat(colors...)), colormap=:jet)
fig, ax, ob = mesh(primal_earth, color=colors[1], colorrange = (-0.0001, 0.0001), colormap=:jet)
Colorbar(fig[1,2], ob)
framerate = 5

# Animation
record(fig, "weather.gif", range(0.0, tₑ; length=150); framerate = 30) do t
    ob.color = findnode(soln(t), :P)
end

LoadError: UndefVarError: tₑ not defined

In [ ]:
using HDF5

#h5open("filename.h5","w") do file
    #write(file, "soln", soln) 
#end

#c = h5open()